In [1]:
from sgkit.distance.api import pairwise_distance
import dask.array as da

from dask.diagnostics import ProgressBar, ResourceProfiler
from bokeh.io import output_notebook

import dask
import fsspec
import zarr


output_notebook()

store = fsspec.get_mapper('gs://ag1000g-release/phase2.AR1/variation/main/zarr/all/ag1000g.phase2.ar1/')
callset_snps = zarr.open_consolidated(store=store)

Loading BokehJS ...

In [2]:
from dask.distributed import Client
dask.config.set({'temporary_directory': '/work/aktech/tmp'})
client = Client(n_workers=8, 
                threads_per_worker=4,
                memory_limit='7GB')
client

Client Scheduler: tcp://127.0.0.1:45827 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 32 Memory: 56.00 GB


In [3]:
gt = callset_snps['2R/calldata/GT']
gt_da = da.from_zarr(gt, chunks=(-1, 1, -1))
x = gt_da[:, :, 1].T
x

dask.array<transpose, shape=(1142, 24767689), dtype=int8, chunksize=(1, 24767689), chunktype=numpy.ndarray>

In [4]:
_x = x
_x

dask.array<transpose, shape=(1142, 24767689), dtype=int8, chunksize=(1, 24767689), chunktype=numpy.ndarray>

In [ ]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    out = pairwise_distance(_x)

/home/aktech/sgkit/sgkit/distance/api.py:91: PerformanceWarning: Increasing number of chunks by factor of 1142
  x_distance = da.blockwise(
